# Programming Exercise 9: scikit-learn exercise

## Introduction

In this exercise we will investigate the [`scikit-learn`](https://scikit-learn.org/stable/) library. We will start by predicting breast cancer, before we continue to predict digits in the MNIST dataset. The remaining exercises in this course will to some extent resemble tutorials, compared to the previous exercises. That is, much of the mathematics are skipped, and we will now primarily look at software packages and examples of empoying machine learning.

In [ ]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np
np.random.seed(42)  # Set the global random seed to make reproducible experiments (scikit-learn also use this)

# Efficient and easy handling of data
import pandas as pd

# Python plotting library
import matplotlib.pyplot as plt

# Plotting tool based on matplotlib for fast visualization of data
import seaborn as sns

# Load function to plot confusion matrices
from utils import plot_confusion_matrix
 
# Machine learning tools
from sklearn.datasets import load_digits                     # MNIST dataset
from sklearn.preprocessing import StandardScaler             # Feature scaling
from sklearn.model_selection import train_test_split         # Split data into train and test set
from sklearn.metrics import classification_report            # Summary of classifier performance
from sklearn.metrics import confusion_matrix                 # Confusion matrix
from sklearn.metrics import accuracy_score

# Time training and prediction time
from time import time

# Tell matplotlib to plot directly in the notebook
%matplotlib inline  

# Autoreload every time a cell is run
%load_ext autoreload
%autoreload 2

## 1 Predicting breast cancer
In this part of the exercise we will look at breast cancer prediction using various machine learning classifiers. We will start by loading the data into Pandas to investigate them, before they are loaded into NumPy to follow the line of the previous exercises. Then we will try a variety of classifiers, compare them, and discuss pros/cons.

### 1.1 Investigate the data

This part will demonstrate some of the advantages whem employing Pandas and Seaborn to investigate your data. We have not been using these libraries in this course, as the main emphasis should be on machine learning. However, if you would like to master a very strong toolbox for applying machine learning using Python, it is advised to learn how to use these libraries. There is no code to be implemented in this part of the exercise, which should rather be seen as a demonstration of how these libraries can help you gain an overview of your data.

In [ ]:
# Load the data into Pandas (df is an abbreviation for dataframe)
df_cancer = pd.read_csv(os.path.join('Data', 'breast_cancer.csv'))

# Show the first 5 examples, including values for all input features.
df_cancer.head(5)

We now have an overview of the input data. However, there are two columns, 'id' and 'Unnamed: 32' which does not contain valuable information, and will therefore be dropped.

In [ ]:
# Remove unnecessary columns
df_cancer.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)

Now let us look at the number of samples in the two classes.

In [ ]:
# Count number of samples in each class
total = df_cancer['diagnosis'].count() # Count total number of samples
malignant = df_cancer[df_cancer['diagnosis'] == "M"]['diagnosis'].count() # Count how many has diagnosis = M (for malignent)
print("Malignant: ", malignant)
print("Benign: ", total - malignant)

So the classes are not perfectly balanced (ie. they do not have the same number of samples in each class). However, they are not not skewed enough for us to take further action in this exercise. If you would like to see an example on how to deal with imbalanced classes when training an SVM classifier, look here: https://scikit-learn.org/stable/auto_examples/svm/plot_separating_hyperplane_unbalanced.html. 

Pandas can automatically generate statistics based on the data in the dataframe. In the following cells we will quickly look at statistics for the entire dataset, then statistics for the malignent cases, and finally statistics for the benign cases.

In [ ]:
# Generate statistics for entire dataset
df_cancer.describe()

In [ ]:
# Generate statistics for malignent cases
df_cancer[df_cancer['diagnosis'] == "M"].describe()

In [ ]:
# Generate statistics for bening cases
df_cancer[df_cancer['diagnosis'] == "B"].describe()

We already see some differences from these statistics. For instance, the mean value of 'radius_mean' differ quite significantly between the malignent and benign cases (17.46 and 12.15 respectively). So we start to get a brief understanding of the data. To go a step further, we will do a scatterplot matrix using the Seaborn library. You can read more about scatteplot matrices here: https://seaborn.pydata.org/examples/scatterplot_matrix.html.

In [ ]:
with sns.plotting_context(rc={"axes.labelsize":16}):  # Temporarily change the font size for seaborn plots
    # Select the first 8 columns 
    df_cancer_plot_features = df_cancer.iloc[:, :9]
    
    # Create pairgrid with hue set to show the two different diagnoses
    g = sns.PairGrid(df_cancer_plot_features, hue="diagnosis", palette="Set2")
    g = g.map_offdiag(plt.scatter, edgecolor="w", s=40)

    # Show histograms of the data on the diagonal and plot the figure
    g = g.map_diag(plt.hist, edgecolor="w")
    plt.show()

This looks promising for a classifier to perform well. However, we do see some redundant features, where the 'radius_mean' and 'perimeter_mean' are simply scaled versions of each other (which makes perfect sense). Having both of these features in the dataset might be relevant to a doctor, but should be avoided when employing machine learning classifiers. In this case, we will only remove the 'perimeter_mean' features. It should be noted that this would automatically be fixed by employing dimensionality reduction, which we have completely skipped in this course, as we needed to save time such that we could get to deep learning. However, Lecture 14 and Exercise 7 from Andrew Ng's Machine learning course cover dimensionality reduction, if you would like to investigate it further.

In [ ]:
# Remove redundant features
df_cancer.drop(['perimeter_mean'], axis=1, inplace=True)

In Seaborn, we can also generate correlation matrices. Let us do this for the first 16 columns. 

In [ ]:
# Select the first 16 columns 
df_cancer_plot_features = df_cancer.iloc[:, :17]

# Calculate correlation coefficients
# (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html)
df_cancer_corr = df_cancer_plot_features.corr()

# Create figure and plot the correlation coefficients as a heatmap
plt.figure(figsize=(15, 15))
sns.heatmap(df_cancer_corr, cmap="Blues", annot=True)
plt.show()

### 1.2 Pre-process the data

Now we have a reasonably good overview of the data we are working with. Before we can start training classifiers, however, we need to map the two classes to numerical values

In [ ]:
# Assign the classes numerical values
df_cancer['diagnosis'] = df_cancer['diagnosis'].map({'M':1,'B':0}) # Sets M = 1 and B = 0½

Now let us go back to using numpy for the remainder of the breast cancer exercise

In [ ]:
array = df_cancer.values

# Define the independent variables as features.
X = array[:,1:]

# Define the target (dependent) variable as labels.
y = array[:,0]

# Store the class names for later to plot results
class_names = ['Malignent', 'Benign']

Is is important to split the data into training and test data. To do this, look at the documentation for the 'train_test_split()' function in scikit learn: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html. Now provide the function in the cell below with the arguments to make a 70/30 split between train and test data respectively. Note that the random state was already set in the top of the notebook, where we defined random seed for the NumPy library, which is also what scikit learn uses for random generators.

In [ ]:
# ====================== YOUR CODE HERE =======================

# Create a train/test split using 30% test size.
X_train, X_test, y_train, y_test = train_test_split()  # You need to provide the function the correct arguments

# =============================================================

# Check the split by printing the shape of each set.
print("X_train shape: " + str(X_train.shape))
print("y_train shape: " + str(y_train.shape))
print("X_test shape:  " + str(X_test.shape))
print("y_test shape:  " + str(y_test.shape))

The following function evaluates a classifier. It first fits the classifier to the training data, before it uses the trained classifier to classify the test data. If feature scaling is desired, this will be carried out as the first step.

In [ ]:
def evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=False):
    """
    This function evaluates a classifier. It measure training and prediction time, and 
    prints performance metrics and a confustion matrix. The returned classifier and 
    scaler are fitted to the training data, and can be used to predict new samples.
    """
    
    # Perform feature scaling
    scaler = StandardScaler()  # Scale to mean = 0 and std_dev = 1
    if feature_scale:
        # ====================== YOUR CODE HERE =======================

        # Hint: Use the newly created scaler object.
        #       See more here: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

        # =============================================================
        
    # Store the time so we can calculate training time later
    t0 = time()

    # Fit the classifier on the training features and labels
    # ====================== YOUR CODE HERE =======================



    # =============================================================
    
    # Calculate and print training time
    print("Training time:", round(time()-t0, 4), "s")

    # Store the time so we can calculate prediction time later
    t1 = time()
    
    # Use the trained classifier to classify the test data (X_test)
    # ====================== YOUR CODE HERE =======================

    predictions = 

    # =============================================================
    
    # Calculate and print prediction time
    print("Prediction time:", round(time()-t1, 4), "s")

    # Evaluate the model
    train_accuracy = clf.score(X_train, y_train)
    test_accuracy = clf.score(X_test, y_test)
    report = classification_report(y_test, predictions, target_names=class_names)

    # Print the reports
    print("\nReport:\n")
    print("Train accuracy: {}".format(round(train_accuracy, 4)))
    print("Test accuracy: {}".format(round(test_accuracy, 4)))
    print("\n", report)
    
    # Plot confusion matrices
    cnf_matrix = confusion_matrix(y_test, predictions)
    plot_confusion_matrix(cnf_matrix, classes=class_names)
    
    # Return the trained classifier to be used on future predictions
    return clf, scaler

Now let us try the implemented function by evaluating logistic regression.

In [ ]:
from sklearn.linear_model import LogisticRegression          

# Instantiate classifier.
clf = LogisticRegression(solver='newton-cg')

# Evaluate classifier without feature scaling
clf_trained, _ = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=False)

You should get accuracies of 0.9548 and 0.9708 for the train and test set respectively. Those are already very good results. Note that the normalized confusion matrix is normalized with respect to each row (each true label).

Let us test the classifier evaluation function when employing feature scaling.

In [ ]:
# Instantiate classifier.
clf = LogisticRegression(solver='newton-cg')

# Evaluate classifier with feature scaling
clf_trained, scaler_fitted = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names, feature_scale=True)

You should get accuracies of 0.9874 and 0.9825 for the train and test set respectively.

### 1.3 Evaluate classifiers
In this section you will try a variety of classifiers. Try looking at the following link for inspiration: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html. It contains a comparison of various classifiers, with decision boundaries plotted to provide some intuition as to how they work.

As you go along in this exercise, remember to interpret the results, and see if you can intuitively understand what they mean.

#### 1.3.1 Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier              

# Instantiate and evaluate classifier
clf = DecisionTreeClassifier()
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=False)

#### 1.3.2 Linear Support Vector Machine (SVM)

In [ ]:
from sklearn.svm import SVC   

# Instantiate and evaluate classifier
clf = SVC(kernel='linear')
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=True)

#### 1.3.3 Kernel based Support Vector Machine (SVM)

In [ ]:
# Instantiate and evaluate classifier
clf = SVC(kernel='rbf')
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=True)

#### 1.3.4 Multilayer Perceptron (MLP) - also referred to as fully connected neural network

In [ ]:
from sklearn.neural_network import MLPClassifier

# Instantiate and evaluate classifier
clf = MLPClassifier(solver='lbfgs', alpha=10, hidden_layer_sizes=(25, 25))  # See what happens when you change random state
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=True)

Now try your own classifiers, and remember to compare the relevant metrices. For instance, a high recall value for the malignent tumors are extremely important in the case of cancer prediction. If you are not unsure as to why this is, try looking here: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9.

### 1.4 Brief look at k-fold cross-validation

We have not covered k-fold cross-validation in these exercises, although it is a highly relevant, especially if you have small datasets. If you are unsure of how this works, take a look here: https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation. Regarding how it is performed in scikit-learn, look here: https://scikit-learn.org/stable/modules/cross_validation.html. Below is an example of how straight-forward it is to employ 10-fold cross-validation on the breast cancer dataset using scikit-learn.

In [ ]:
from sklearn.model_selection import cross_val_score

# Use 10-fold cross-validation to evaluate logistic regression on our dataset
clf = LogisticRegression(solver='newton-cg')
scores = cross_val_score(clf, X, y, cv=10)  # Remember to provide the entire dataset

print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))

Looking at the results from the 10-fold cross-validation, we can conclude that logistic regression results in a high accuracy, and that for the ten classifiers we trained, the accuracy remained consistent. 

One additional note regarding scikit-learn is on hyperparameter optimization. We will not investigate it here, but if you want to, e.g., find the optimal number of neurons in the hidden layer of your neural network classifier, scikit-learn has built-in methods to help you. Take a look here: https://scikit-learn.org/stable/modules/grid_search.html.

A final note regarding the possibilities in scikit-learn should be on pipelining. This is a method for coupling several classifiers, or pre-processing and classification steps, into one single function call. If you would like to learn more, take a look here: https://towardsdatascience.com/a-simple-example-of-pipeline-in-machine-learning-with-scikit-learn-e726ffbb6976.

## 2 MNIST Classification
### 2.1 Get the data

Load the MNIST data from Exercise 4. 

In [ ]:
# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

#  training data stored in arrays X, y
data = loadmat(os.path.join('Data', 'ex4data1.mat'))
X, y = data['X'], data['y'].ravel()

# set the zero digit to 0, rather than its mapped 10 in this dataset
# This is an artifact due to the fact that this dataset was used in 
# MATLAB where there is no index 0
y[y == 10] = 0

# Number of training examples
m = y.size

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Define class names as 'Class 0', 'Class 1' ... 'Class 9'
class_names = []
for i in range(10):
    class_names.append('Class ' + str(i))

### 2.2 Evaluate various classifiers
Let us evaluate several classifiers on the MNIST dataset. Remember to pay attention to the test scores, and to the training and prediction times. Often, we do not care too much about a high training time, as it is not critical once the classifier is deployed for usage. A high prediction time can be very computationally expensive if you have thousands of users which employ your classifier.

#### 2.2.1 Neural network from Exercise 4

Let us try recreating the neural networks architecture which we made in Exercise 4 using scikit-learn.

In [ ]:
# Instantiate and evaluate classifier
clf = MLPClassifier(solver='lbfgs', alpha=1, hidden_layer_sizes=(25, 10))
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=True)

We see that the test set accuracy is quite low, compared to the training set accuracy. Try finding a good 'alpha' parameter (L2 regularization parameter) to maximize the test set accuracy.

#### 2.2.2 Kernel based SVM
Let us now try a kernel based SVM. Remember how we can use the One-vs-Rest method to use binary classifiers in multi-class problems.

In [ ]:
from sklearn.multiclass import OneVsRestClassifier

# Instantiate and evaluate classifier
clf = OneVsRestClassifier(SVC(kernel='rbf', gamma='scale'))
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=True)

#### 2.2.3 Logistic regression

In [ ]:
# Instantiate and evaluate classifier
clf = OneVsRestClassifier(LogisticRegression(solver='newton-cg'))
clf_trained = evaluate_classifer(clf, X_train, X_test, y_train, y_test, class_names,  feature_scale=True)

We see how easy it is to train various classifiers, and we see how important it is to always evaluate our classifiers on a test dataset.

### 2.3 Full size MNIST dataset (you need TensorFlow and Keras)

The MNIST dataset from Exercise 4 only contain 5.000 samples, and each sample is only 20x20 pixels. The full size MNIST dataset contains 60.000 training samples and 10.000 test samples, with each sample being 28x28 pixels. 

In [ ]:
import keras
from keras.datasets import mnist

# the data, split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

Now try varying the number of samples you want to use, and see how this affects the training time. If you want to compare the training time here with the one from the neural network trained using the MNIST data from Exercise 4 in section 2.2.1, then use 3500 training samples and 1500 test samples. This is equivalent to the 70/30 split on the 5000 total number of samples we used at that point. Doing this will let you compare the training time when using 20x20 pixel images and 28x28 pixels images.

Then try to find the optimally performing MLP classifier on the MNIST data. Vary alpha, hidden_layer_sizes, and try looking up the documentation for the MLPClassifier to see if there are more interesting things to adjust.

In [ ]:
# Define the number of train and test samples you want to use
train_samples_to_use = 3500  # Max. is 60000
test_samples_to_use = 1500  # Max. is 10000

# Instantiate and evaluate classifier
clf = MLPClassifier(solver='lbfgs', alpha=30, hidden_layer_sizes=(25, 10))
clf_trained = evaluate_classifer(clf,
                                 X_train[0:train_samples_to_use-1, :],
                                 X_test[0:test_samples_to_use-1, :],
                                 y_train[0:train_samples_to_use-1],
                                 y_test[0:test_samples_to_use-1],
                                 class_names,
                                 feature_scale=True)

As we see, the training time increases quite significantly compared to section 2.2.1, even though we just went from 20x20 pixels in the MNIST data we used in exercise 4, to the 28x28 pixels in the full size MNIST data. This signifies one major issue with these methods. Imagine that instead of training on the 28x28 pixels images here, we have FullHD imagery (1920x1080 pixels). As we will see, an issue which we have not discussed yet have also arisen. The classifiers are highly spatially variant. That is, if we were to shift the digits in the image slightly away from the center, the classifier breaks down. We will solve this obstacle next time, and simultaneously improve accuracy, by using convolutional neural networks. 